In [1]:
import numpy as np
import pandas as pd
import zipfile as zfile
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
from tradeanalysis import TradeReport as trr
from tradefile import TradeFile as trf

In [ ]:
data_2020_2021 = trf("../data/import_PC_2020-2020.zip", merge_file="../data/import_PC_2021-2021.zip")
print(data_2020_2021.data.describe())
print(data_2020_2021.data.sample(10))

In [ ]:
data_2020_2021.save_to_file(path="../data/")

In [2]:
report = trr(source_file="../data/HS_2016-01_2021-04.csv")

TradeReport: the uploaded data start at date 2016-01-01 00:00:00 and end at date 2021-04-01 00:00:00.


In [3]:
report.trade_df.head()

value
kind date       country code      unit             
HS   2016-01-01 103     000000011 JPY       1424000
                                  KG           5003
                        000000019 JPY       2969000
                                  KG           3744
                        000000099 JPY   12351184000

In [ ]:
yoy_report = report.yoy_country_report(kind='PC', country=220)

In [ ]:
yoy_report.head(10)

In [4]:
start = report.last_date - pd.DateOffset(months=23)

In [9]:
start

Timestamp('2019-03-01 00:00:00')

In [5]:
end = report.last_date

In [11]:
end

Timestamp('2021-02-01 00:00:00')

In [ ]:
report.trade_df.head(10)

In [ ]:
report.trade_df.loc['PC', :, 220, '0070101', 'JPY']

# Comparison report

In [ ]:
drilled = report.trade_df.loc['PC', start:end, [210,220], '1010103', 'JPY']

In [ ]:
date_ix = pd.date_range(start, end, freq=pd.offsets.MonthBegin(), closed=None)
date_ix

In [ ]:
drilled.index.get_level_values(2).unique()

In [ ]:
new_mux = pd.MultiIndex.from_product([drilled.index.get_level_values(2).unique(), date_ix], names=['country', 'date'])
trends = drilled.groupby(['country', 'date']).mean().reindex(new_mux, fill_value=0)
trends.reset_index().pivot(index='date', columns='country', values='value').plot(figsize=(10,7))
plt.show()

# Year-on-Year report

In [6]:
code_mask = report.trade_df.index.get_level_values('code').str.len() > 3
drilled_yoy = report.trade_df.loc['HS', start:end, 220, code_mask, :].copy()
print(drilled_yoy.head(5))
year_sums = drilled_yoy.droplevel(
    ['kind', 'country']).groupby(['code'] +
                                 [pd.Grouper(freq='12M', level='date', closed='left')] +
                                 ['unit']).sum()
year_compared = year_sums.reset_index().pivot(index='code', columns=['date', 'unit'], values='value')
year_compared = year_compared.sort_values(by=(year_compared.columns.get_level_values(0).max(), 'JPY'), ascending=False)
year_compared.head(20)

                                            value
kind date       country code      unit           
HS   2019-05-01 220     000000011 JPY     8393000
                                  KG        68342
                        000000019 JPY    13231000
                                  KG        10664
                        000000099 JPY   231215000


date      2020-04-30                                                       \
unit               L CM CT      DZ       GR           JPY        KG KL MT   
code                                                                        
240399200          0  0  0       0        0  191664430000  18281319  0  0   
870324000          0  0  0       0        0   61819722000         0  0  0   
300490029          0  0  0       0        0   45288312000    478483  0  0   
420291000          0  0  0   71458        0   30698788000    715027  0  0   
711319029          0  0  0       0  4050708   29324476000         0  0  0   
420231200          0  0  0  148784        0   26457919000    379089  0  0   
300490024          0  0  0       0        0   28371583000    217881  0  0   
420292000          0  0  0   61616        0   18970603000    472989  0  0   
293359400          0  0  0       0        0   16263540000     63143  0  0   
841191000          0  0  0       0        0   20707875000    164105  0  0   
220421020   34969140  0  0       0        0   17628741000         0  0  0   
840820000          0  0  0       0        0   15693323000   8282316  0  0   
300215000          0  0  0       0        0    9724983000      2371  0  0   
420221210          0  0  0       0        0   18412158000    295412  0  0   
851770000          0  0  0       0        0     495066000     10634  0  0   
870323000          0  0  0       0        0   14560697000         0  0  0   
200210000          0  0  0       0        0    8867736000  92363810  0  0   
870322000          0  0  0       0        0   11913085000         0  0  0   
190219093          0  0  0       0        0    8192467000  71672119  0  0   
150910000          0  0  0       0        0   11010203000  16259443  0  0   

date               ... 2021-04-30                                              \
unit           NO  ...         GR           JPY        KG KL MT      NO PR SM   
code               ...                                                          
240399200       0  ...          0  202789158000  17534036  0  0       0  0  0   
870324000    2124  ...          0   49117749000         0  0  0    1598  0  0   
300490029       0  ...          0   35624048000    388014  0  0       0  0  0   
420291000       0  ...          0   25554190000    568759  0  0       0  0  0   
711319029       0  ...    3225012   23849261000         0  0  0       0  0  0   
420231200       0  ...          0   23231725000    338910  0  0       0  0  0   
300490024       0  ...          0   18742652000    238257  0  0       0  0  0   
420292000       0  ...          0   18328806000    439258  0  0       0  0  0   
293359400       0  ...          0   16633239000     83562  0  0       0  0  0   
841191000       0  ...          0   16137327000    117601  0  0       0  0  0   
220421020       0  ...          0   15475468000         0  0  0       0  0  0   
840820000   32259  ...          0   15286878000   7837156  0  0   30393  0  0   
300215000       0  ...          0   13045515000      2655  0  0       0  0  0   
420221210  398085  ...          0   12624602000    199258  0  0  283223  0  0   
851770000       0  ...          0   11438543000     48944  0  0       0  0  0   
870323000    2312  ...          0   11179644000         0  0  0    2279  0  0   
200210000       0  ...          0   10335404000  94418329  0  0       0  0  0   
870322000    5349  ...          0    9951442000         0  0  0    4504  0  0   
190219093       0  ...          0    9374812000  75958755  0  0       0  0  0   
150910000       0  ...          0    9365402000  15722568  0  0       0  0  0   

date             
unit      ST TH  
code             
240399200  0  0  
870324000  0  0  
300490029  0  0  
420291000  0  0  
711319029  0  0  
420231200  0  0  
300490024  0  0  
420292000  0  0  
293359400  0  0  
841191000  0  0  
220421020  0  0  
840820000  0  0  
300215000  0  0  
420221210  0  0  
851770000  0  0  
870323000  0  0  
200210000  0  0  
8703

In [8]:
codes = pd.read_csv("../data/HS_codes.csv", sep=',', dtype={'HS': 'str'})

In [9]:
codes_ix = codes.set_index('HS')

codes_ix.columns = [[pd.to_datetime(0)], ['Description']]
codes_ix.head()

,1970-01-01
,Description
HS,
000000011,Aircrafts'Stores(Ex.Fuels Etc)F/Food Etc
000000019,"Aircrafts'Stores(Ex.Fuels Etc.), N.E.S."
010121100,"Live Horses, Pure-Bred (Certified)"
010121210,"Live Horses, Pure-Bred, L-Breed(Certified)"
010121290,"Live Horses, Pure-Bred, N.E.S."


In [10]:
country_report_1 = codes_ix.merge(year_compared, left_index=True, right_index=True)

country_report_1.head()

1970-01-01 2020-04-30              \
                                        Description          L CM CT DZ GR   
000000011  Aircrafts'Stores(Ex.Fuels Etc)F/Food Etc          0  0  0  0  0   
000000019   Aircrafts'Stores(Ex.Fuels Etc.), N.E.S.          0  0  0  0  0   
010129290                       Live Horses, N.E.S.          0  0  0  0  0   
010619090                       Live Mammals N.E.S.          0  0  0  0  0   
010620010                              Live Turtles          0  0  0  0  0   

                                    ... 2021-04-30                            \
                 JPY      KG KL MT  ...         GR      JPY     KG KL MT  NO   
000000011   49549000  388945  0  0  ...          0  2263000  35489  0  0   0   
000000019  467811000   75164  0  0  ...          0  1434000   1803  0  0   0   
010129290   17665000       0  0  0  ...          0        0      0  0  0   0   
010619090          0       0  0  0  ...          0   215000      6  0  0   2   
010620010    2062000       8  0  0  ...          0  1191000     10  0  0  42   

                       
          PR SM ST TH  
000000011  0  0  0  0  
000000019  0  0  0  0  
010129290  0  0  0  0  
010619090  0  0  0  0  
010620010  0  0  0  0  

[5 rows x 29 columns]

In [19]:
country_report_1.sort_values(by=(year_compared.columns.get_level_values(0).max(), 'JPY'),
                             ascending=False).to_csv('../data/country_report_italy.csv')

In [16]:
country_report_1.head(5)

1970-01-01 2020-04-30              \
                                        Description          L CM CT DZ GR   
000000011  Aircrafts'Stores(Ex.Fuels Etc)F/Food Etc          0  0  0  0  0   
000000019   Aircrafts'Stores(Ex.Fuels Etc.), N.E.S.          0  0  0  0  0   
010129290                       Live Horses, N.E.S.          0  0  0  0  0   
010619090                       Live Mammals N.E.S.          0  0  0  0  0   
010620010                              Live Turtles          0  0  0  0  0   

                                    ... 2021-04-30                            \
                 JPY      KG KL MT  ...         GR      JPY     KG KL MT  NO   
000000011   49549000  388945  0  0  ...          0  2263000  35489  0  0   0   
000000019  467811000   75164  0  0  ...          0  1434000   1803  0  0   0   
010129290   17665000       0  0  0  ...          0        0      0  0  0   0   
010619090          0       0  0  0  ...          0   215000      6  0  0   2   
010620010    2062000       8  0  0  ...          0  1191000     10  0  0  42   

                       
          PR SM ST TH  
000000011  0  0  0  0  
000000019  0  0  0  0  
010129290  0  0  0  0  
010619090  0  0  0  0  
010620010  0  0  0  0  

[5 rows x 29 columns]

In [90]:
fin_report = country_report_1.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [70]:
fin_report.head()

L                                      CM  \
          2020-04-30 00:00:00 2021-04-30 00:00:00 2020-04-30 00:00:00   
000000011                   0                   0                   0   
000000019                   0                   0                   0   
010129290                   0                   0                   0   
010619090                   0                   0                   0   
010620010                   0                   0                   0   

                                               CT                      \
          2021-04-30 00:00:00 2020-04-30 00:00:00 2021-04-30 00:00:00   
000000011                   0                   0                   0   
000000019                   0                   0                   0   
010129290                   0                   0                   0   
010619090                   0                   0                   0   
010620010                   0                   0                   0   

                           DZ                      \
          2020-04-30 00:00:00 2021-04-30 00:00:00   
000000011                   0                   0   
000000019                   0                   0   
010129290                   0                   0   
010619090                   0                   0   
010620010                   0                   0   

                                        Description                  GR  ...  \
                                1970-01-01 00:00:00 2020-04-30 00:00:00  ...   
000000011  Aircrafts'Stores(Ex.Fuels Etc)F/Food Etc                   0  ...   
000000019   Aircrafts'Stores(Ex.Fuels Etc.), N.E.S.                   0  ...   
010129290                       Live Horses, N.E.S.                   0  ...   
010619090                       Live Mammals N.E.S.                   0  ...   
010620010                              Live Turtles                   0  ...   

                          JPY        KG        KL        MT        NO  \
          variation variation variation variation variation variation   
000000011         0         0         0         0         0         0   
000000019         0         0         0         0         0         0   
010129290         0         0         0         0         0         0   
010619090         0         0         0         0         0         0   
010620010         0         0         0         0         0         0   

                 PR        SM        ST        TH  
          variation variation variation variation  
000000011         0         0         0         0  
000000019         0         0         0         0  
010129290         0         0         0         0  
010619090         0         0         0         0  
010620010         0         0         0         0  

[5 rows x 43 columns]

In [91]:
for col in fin_report.columns.levels[0]:
    if col != 'Description':
        fin_report[col, 'variation'] = ((1 + fin_report[col, year_compared.columns.get_level_values(0).max()]) / \
                                        (1 + fin_report[col, year_compared.columns.get_level_values(0).min()])) - 1

                            L                                      CM  \
          2020-04-30 00:00:00 2021-04-30 00:00:00 2020-04-30 00:00:00   
000000011                   0                   0                   0   
000000019                   0                   0                   0   
010129290                   0                   0                   0   
010619090                   0                   0                   0   
010620010                   0                   0                   0   

                                               CT                      \
          2021-04-30 00:00:00 2020-04-30 00:00:00 2021-04-30 00:00:00   
000000011                   0                   0                   0   
000000019                   0                   0                   0   
010129290                   0                   0                   0   
010619090                   0                   0                   0   
010620010                   0                   0 

In [92]:
fin_report = fin_report.sort_index(axis=1)

In [101]:
fin_report[['Description', 'JPY', 'NO']].sort_values(by=('JPY', year_compared.columns.get_level_values(0).max()),
                             ascending=False).drop_duplicates().head(30)

Description                 JPY  \
                                  1970-01-01 00:00:00 2020-04-30 00:00:00   
240399200   Manufactured Tobacco & Substitutes, N.E.S        191664430000   
870324000           Motor Cars & Other Motor Vehicles         61819722000   
300490029                         Medicaments, N.E.S.         45288312000   
420291000     Article Of Cases, Outer Surface Leather         30698788000   
711319029   Art.Of Jewellery & P/T Of P-Metal, N.E.S.         29324476000   
420231200     Cigarette-Case & Similar, Outer Leather         26457919000   
300490024       Medicaments, N.E.S. (For Retail Sale)         28371583000   
420292000       Article Of Cases Pla.-Sheet & Textile         18970603000   
293359400    Compound Of Cont.Pyrimidine Ring.Etc(Inn         16263540000   
293359400       Compounds Of Cont.Pyrimidine Ring.Etc         16263540000   
841191000         P/T Of Turbo-Jet Or Turbo-Propeller         20707875000   
220421020                     Wine, In Containers=<2L         17628741000   
840820000      Diesel Eng. F/Vehicle Of Chapter No.87         15693323000   
300215000     Measured Immunological Products, Retail          9724983000   
420221210         Handbags, Leather Or Patent Leather         18412158000   
851770000                   P/T Of Communication App.           495066000   
870323000           Motor Cars & Other Motor Vehicles         14560697000   
200210000  Tomato(Whole, Pieces), Prepared, Preserved          8867736000   
870322000           Motor Cars & Other Motor Vehicles         11913085000   
190219093                                   Spaghetti          8192467000   
150910000                           Olive Oil, Virgin         11010203000   
293299000    Heterocyclic Compound With Oxygen, N.E.S         10425624000   
420221110     Handbags >Y6000, With P.Metals, Leather         11336994000   
300420000     Antibiotics Preparation, N.E.S.(Retail)          8479201000   
390720990                      Polyether, N.E.S.(Inn)          5762366000   
390720990                           Polyether, N.E.S.          5762366000   
180632100   Chocolate, Not Filled (Block, Slab & Etc)          4225465000   
854140090   Photosensitive Semiconductor Devices, Nes          9005078000   
293399090    Heterocyclic Comp W/Nitrogen, N.E.S(Inn)          4668383000   
293399090    Heterocyclic Compounds W/Nitrogen, N.E.S          4668383000   

                                                          NO  \
          2021-04-30 00:00:00  variation 2020-04-30 00:00:00   
240399200        202789158000   0.058043                   0   
870324000         49117749000  -0.205468                2124   
300490029         35624048000  -0.213394                   0   
420291000         25554190000  -0.167583                   0   
711319029         23849261000  -0.186711                   0   
420231200         23231725000  -0.121937                   0   
300490024         18742652000  -0.339386                   0   
420292000         18328806000  -0.033831                   0   
293359400         16633239000   0.022732                   0   
293359400         16633239000   0.022732                   0   
841191000         16137327000  -0.220715                   0   
220421020         15475468000  -0.122146                   0   
840820000         15286878000  -0.025899               32259   
300215000         13045515000   0.341443                   0   
420221210         12624602000  -0.314333              398085   
851770000         11438543000  22.105087                   0   
870323000         11179644000  -0.232204                2312   
200210000         10335404000   0.165507                   0   
870322000          9951442000  -0.164663                5349   
190219093          9374812000   0.144321                   0   
150910000          9365402000  -0.149389                   0   
293299000          8839635000  -0.152124                   0   
420221110          7437115000  -0.343996 